In [14]:
import torch
from torch import nn
from torch import optim
import numpy as np

In [4]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.dataset import random_split

In [7]:
num_workers = 0
batch_size = 20

transform = transforms.ToTensor()

train_data = datasets.MNIST(root='./pytorch_실습/data',train = True, download = True, transform = transform)
train_data, valid_data = random_split(train_data,[50000,10000])
test_data = datasets.MNIST(root='./pytorch_실습/data',train = False, download = True, transform = transform)

In [8]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [10]:
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        hidden1 = 512
        hidden2 = 512
        
        self.fc1 = nn.Linear(28*28, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2,10)
        
    def forward(self, x):
        x = x.view(-1,28*28)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        outputs = self.fc3(x)
        return outputs
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.05, momentum=0.9)
epochs = 10

In [16]:
valid_loss_min = np.Inf
for epoch in range(epochs):
    train_loss = 0.
    valid_loss = 0.
    model.train()
    for x,y in train_loader :
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs,y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for x,y in valid_loader :
        outputs = model(x)
        loss = criterion(outputs,y)
        valid_loss += loss.item()
        
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(valid_loader)
    
    print('Epoch: {} \tTraining Loss: {:.6f}\tValidation Loss: {:.6f}'.format(
        epoch+1,train_loss,valid_loss
    ))
    
    if valid_loss <= valid_loss_min :
        print('Validation loss decreased ([{:.6f} -->{:.6f}]). saving model ...'. format(
        valid_loss_min, valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.130591	Validation Loss: 0.159845
Validation loss decreased ([inf -->0.159845]). saving model ...
Epoch: 2 	Training Loss: 0.096796	Validation Loss: 0.104747
Validation loss decreased ([0.159845 -->0.104747]). saving model ...
Epoch: 3 	Training Loss: 0.078208	Validation Loss: 0.127935
Epoch: 4 	Training Loss: 0.074148	Validation Loss: 0.107574
Epoch: 5 	Training Loss: 0.059599	Validation Loss: 0.128159
Epoch: 6 	Training Loss: 0.057636	Validation Loss: 0.211638
Epoch: 7 	Training Loss: 0.058265	Validation Loss: 0.115218
Epoch: 8 	Training Loss: 0.050603	Validation Loss: 0.133786
Epoch: 9 	Training Loss: 0.054982	Validation Loss: 0.112834
Epoch: 10 	Training Loss: 0.045656	Validation Loss: 0.150777


In [17]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [ ]:
test_loss = 0.0
class_correct = [0 for i in range(10)]
class_total = [0. for i in range(10)]

model.eval()

for x, y in test_loader :
    output = model(x)
    loss = criterion(output,y)
    test_loss += loss.item()
    _, preds = torch.max(output,1)
    
    correct = np.squeeze(preds.eq(y.data.view_as(preds)))
    
    #정답에는 1이 들어있고 오답에는 0이 들어있다.
    for i in range(len(y)):
        #label : 정답 index
        label = y.data[i]
        class_correct[label] +=correct[i].item()
        #correct[i].item() 은 0 (오답) 또는 1 (정답)
        class_total[label] += 1
    
test_loss = test_loss/len(test_loader)